# Ball Challenge

In [1]:


import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn
import torch
import functools
from utils import PositionPrediction

plt.style.use("seaborn-v0_8-whitegrid")

from ballchallenge.training import run_training_for_position
from elasticai.creator.file_generation.on_disk_path import OnDiskPath
from elasticai.creator.vhdl.system_integrations.firmware_env5 import FirmwareENv5

GRID_SIZE = (10, 10)
experiment = PositionPrediction(GRID_SIZE)

## Load Training and Test Data

In [2]:

experiment.load_dataset()

print("Train Samples:", len(experiment.train_set))
print("Test Samples:", len(experiment.test_set))

print("Sample Shape:", experiment.input_shape)

Train Samples: 43
Test Samples: 14
Sample Shape: (3, 250)


## Create Model

In [7]:


from itertools import chain
from typing import Callable, Sequence


def bnormed_conv(in_channels, out_channels, kernel_size, act=nn.Sigmoid):
        return nn.Sequential(nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size),
            nn.BatchNorm1d(out_channels),
                             act()
            )

def create_position_model(input_shape: tuple[int, int], kernel_sizes: Sequence[int] = (26, 26, 26), out_channels: Sequence[int]=(3, 3, 3), lin_act: Callable[[], nn.Module] = nn.Hardsigmoid, conv=bnormed_conv, linear_out_features: Sequence[int]=(2,)) -> nn.Module:
    in_length = input_shape[1]
    def flatten_linear(in_channels, out_features, kernel_sizes):
        in_features = in_length
        for k in kernel_sizes:
            in_features -= (k - 1)
        in_features = in_channels * in_features
        return nn.Sequential(nn.Flatten(), nn.Linear(in_features=in_features, out_features=out_features))

    def _conv(in_channels, out_channels, kernel_size):
        return conv(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size)
    convs = []
    convs.append(_conv(in_channels=input_shape[0], out_channels=out_channels[0], kernel_size=kernel_sizes[0]))
    for k, oc, ic in zip(kernel_sizes[1:], out_channels[1:], out_channels[:-1]):
        convs.append(_conv(in_channels=ic, out_channels=oc, kernel_size=k))
    total_num_linears = len(linear_out_features)
    linears = [flatten_linear(in_channels=out_channels[-1], out_features=linear_out_features[0], kernel_sizes=kernel_sizes)]
    if total_num_linears > 1:
        linears.extend([nn.BatchNorm1d(linear_out_features[0]), lin_act()])
    for id, in_features, out_features in zip(range(2, total_num_linears+1), linear_out_features[:-1], linear_out_features[1:]):
        linears.append(nn.Linear(in_features=in_features, out_features=out_features))
        if id < total_num_linears:
            linears.extend([nn.BatchNorm1d(out_features), lin_act()])
    all_layers = tuple(chain(convs, linears))
    return nn.Sequential(
        *all_layers
    )


In [9]:
from itertools import product
import operator
from functools import reduce, partial

# grid search

search_space = {
    'conv_act': [
        # nn.ReLU,
        # nn.Identity,
        nn.Sigmoid,
        # nn.Tanh
    ],
    'lin_act': [
        nn.Identity,
        # nn.Hardtanh,
        # nn.Tanh,
        # nn.Sigmoid,
        # nn.ReLU
    ],
    'conv_setup': [
        # {'out_channels': (3, 1, 1), 'kernel_sizes': (26, 26, 26)},
        # {'out_channels': (3, 3, 3), 'kernel_sizes': (26, 26, 26)},
        # {'out_channels': (3, 3, 3), 'kernel_sizes': (64, 64, 64)},
        # {'out_channels': (1, 1, 1), 'kernel_sizes': (64, 64, 64)},
        {'out_channels': (3, 1, 1), 'kernel_sizes': (64, 26, 13)},
        {'out_channels': (3, 2, 2, 1), 'kernel_sizes': (64, 26, 13, 13)},
        {'out_channels': (3, 2, 2, 1, 1), 'kernel_sizes': (64, 26, 13, 13, 5)}

    ],
    'lin_out_features': [
        (2, ),
    ]
}


def run_grid_search(search_space):

    total_number_of_configurations = reduce(operator.mul, (len(subspace) for subspace in search_space.values()))
    print("total number of configurations ", total_number_of_configurations)

    def expand_search_space(space):
        return [dict(zip(space.keys(), combination)) for combination in product(*tuple(space.values()))]

    tried_configs_and_mean = []
    best_mean = 2
    best_model = None
    trials = 2
    best_trial_id = 0
    for id, config in enumerate(expand_search_space(search_space)):
        for trial in range(trials):
            print(f"trial {2*id+trial} of {total_number_of_configurations*trials}")
            experiment.model = create_position_model(
                input_shape=experiment.input_shape,
                kernel_sizes=config['conv_setup']['kernel_sizes'],
                out_channels=config['conv_setup']['out_channels'],
                lin_act=config['lin_act'],
                linear_out_features=config['lin_out_features'],
                conv=partial(bnormed_conv, act=config['conv_act'])
            )
            experiment.train(900, quiet=True)
            mean, var = experiment.get_mean_and_var_distance_for_test()
            print(mean)
            print(config)

            if mean < best_mean:
                print("new best mean distance (meters) ", mean, "best var ", var)
                print(experiment.model)
                best_mean = mean
                best_model = experiment.model
                best_trial_id = 2*id+trial
            tried_configs_and_mean.append((config, mean, var))
    print("done.")
    print("best model ", best_model)
    print("best mean ", best_mean)
    return best_model, best_trial_id

result = run_grid_search(search_space)
# best config {'conv_act': <class 'torch.nn.modules.activation.Sigmoid'>, 'lin_act': <class 'torch.nn.modules.linear.Identity'>, 'conv_setup': {'out_channels': (3, 2, 2, 1), 'kernel_sizes': (64, 26, 13, 13)}, 'lin_out_features': (2,)}
# new best mean distance (meters)  0.4440311874662127 best var  0.0626205420534494

total number of configurations  3
trial 0 of 6
0.566272208733218
{'conv_act': <class 'torch.nn.modules.activation.Sigmoid'>, 'lin_act': <class 'torch.nn.modules.linear.Identity'>, 'conv_setup': {'out_channels': (3, 1, 1), 'kernel_sizes': (64, 26, 13)}, 'lin_out_features': (2,)}
new best mean distance (meters)  0.566272208733218 best var  0.06968685123610838
Sequential(
  (0): Sequential(
    (0): Conv1d(3, 3, kernel_size=(64,), stride=(1,))
    (1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
  )
  (1): Sequential(
    (0): Conv1d(3, 1, kernel_size=(26,), stride=(1,))
    (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
  )
  (2): Sequential(
    (0): Conv1d(1, 1, kernel_size=(13,), stride=(1,))
    (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
  )
  (3): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear

## Train Model

## Evaluate Trained Model

### Training History

#### On Testing Set

## Save Hardware Implementation

In [5]:
path = OnDiskPath("build")

hw_design = experiment.model.create_design("ball_throw")

channels, signal_length = experiment.input_shape
total_length = channels * signal_length

firmware = FirmwareENv5(
    network=hw_design,
    x_num_values=total_length,
    y_num_values=GRID_SIZE[0] * GRID_SIZE[1],
    skeleton_version="v2",
    id=666
)
firmware.save_to(path)
hw_design.save_to(path)

AttributeError: 'Sequential' object has no attribute 'create_design'

## Additional Section

### Mean Label of the Dataset

In [ ]:
def plot_dataset_mean_label() -> None:
    _, labels = ds[:]
    mean_label = labels.mean(dim=0)
    
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
    sns.heatmap(mean_label.view(GRID_SIZE), square=True, cmap="hot", ax=ax)
    ax.set_title("Mean Label")
    

plot_dataset_mean_label()